In [1]:
import pandas as pd
import json
import geopandas as gpd
import os
%matplotlib inline
import geohash
import geopandas as gp
import pandas as pd
import math
import geojson
from geojson import MultiLineString
from shapely.geometry import MultiLineString
from shapely.geometry import MultiPoint
import numpy as np
from shapely import geometry
from shapely.geometry import Point, Polygon, box,LineString
from geopandas import datasets, GeoDataFrame, read_file
from geopandas.tools import overlay
from matplotlib import pyplot
import matplotlib.pyplot as plt
from shapely.geometry.polygon import LinearRing, Polygon
from numpy import cos,sin,arccos
import numpy as np
from pylab import *
from shapely.ops import cascaded_union
from shapely.geometry import Point, LineString, mapping
import scipy.interpolate
import geopy
import geopy.distance
import time


import networkx as nx

## Reading JSON file

In [117]:
main_arr = []
with open('D:/Datasets/MIMOS/traffic_flow_07092020_2100_2200.json') as f:
    here_traffic_flow = f.readlines()
    for line in here_traffic_flow:
        
        
        line = json.loads(line)
        line = line['_source']
        #print(line)
        for flow_item_s in line['FIS']:
            for flow_item in flow_item_s['FI']:
                #print(flow_item)
                temp_arr = []
                for tmc_item in flow_item['TMC']:
                    #print(flow_item['TMC'][tmc_item])
                    temp_arr.append(flow_item['TMC'][tmc_item])
                
                ## To get SHP
                #shp_values = flow_item['SHP'][0]['value'][0]
                
                arr_lon = []
                arr_lat = []
                
                fc_temp = flow_item['SHP'][0]['FC']
                
                for shp in flow_item['SHP']:
                    shp_values = shp['value'][0]
                    coordinates_arr = shp_values.split(' ')
                    #print(coordinates_arr)
                    
                    for item in coordinates_arr:
                        if len(item) > 0:
                            item = item.split(',')
                            arr_lat.append(float(item[0]))
                            arr_lon.append(float(item[1]))
                            
                    #if fc_temp != shp['FC']:
                        #print(fc_temp, shp['FC'])
                        #print(line['PBT'])
                        #print(line['DE'])
                        #print('FC Diff')
                    
                polygon_geom = LineString(zip(arr_lon, arr_lat))
                #temp_arr.append(flow_item['SHP'][0]['value'])
                temp_arr.append(polygon_geom)
                
                temp_arr.append(flow_item['SHP'][0]['FC'])
                
                
                ## To get all SHP
                """
                for cf_item in flow_item['SHP'][0]:   
                    temp_arr.append(flow_item['SHP'][0][cf_item])
                
                
                for cf_item in flow_item['CF'][0]:   
                    temp_arr.append(flow_item['CF'][0][cf_item])
                """
                try:
                    temp_arr.append(flow_item['CF'][0]['SP'])
                    temp_arr.append(flow_item['CF'][0]['SU'])
                    temp_arr.append(flow_item['CF'][0]['FF'])
                    temp_arr.append(flow_item['CF'][0]['JF'])
                    temp_arr.append(flow_item['CF'][0]['CN'])
                
                    temp_arr.append(line['PBT'])
                    temp_arr.append(line['DE'])
                
                
                
                    #print(len(temp_arr))
                    if len(temp_arr) == 13:
                        main_arr.append(temp_arr)
                except:
                    pass

In [118]:
here_traffic_flow_df = pd.DataFrame(main_arr)

In [119]:
here_traffic_flow_df.columns = ['TMC_PC', 'TMC_DE', 'TMC_QD', 'TMC_LE',
                            'Geometry', 'FC',
                             'CF_SP', 'CF_SU', 'CF_FF', 'CF_JF', 'CF_CN',
                               'PBT', 'DE']



In [120]:
here_traffic_flow_df.tail()

,TMC_PC,TMC_DE,TMC_QD,TMC_LE,Geometry,FC,CF_SP,CF_SU,CF_FF,CF_JF,CF_CN,PBT,DE
353395,23,Jalan Yahya Awal,+,1.04507,"LINESTRING (103.74102 1.48885, 103.7412 1.4888...",2,69.0,69.0,70.0,0.19047,0.7,2020-09-07T21:54:58Z,Jalan Tun Abdul Razak
353396,22,Jalan Kebun Teh,+,0.46699,"LINESTRING (103.74951 1.48519, 103.75043 1.485...",2,64.0,64.0,63.8,0.00000,0.7,2020-09-07T21:54:58Z,Jalan Tun Abdul Razak
353397,21,Jalan Kebun Teh Lama/Jalan Ulu Ayer Molek,+,0.74190,"LINESTRING (103.7531 1.48349, 103.75318 1.4833...",2,63.0,63.0,63.6,0.12578,0.7,2020-09-07T21:54:58Z,Jalan Tun Abdul Razak
353398,20,Jalan Lingkaran Dalam,+,0.79558,"LINESTRING (103.75361 1.47716, 103.75374 1.476...",2,59.0,59.0,59.5,0.11204,0.7,2020-09-07T21:54:58Z,Jalan Tun Abdul Razak
353399,19,3/Jalan Tebrau/Jalan Wong Ah Fook,+,0.71191,"LINESTRING (103.75683 1.47074, 103.75692 1.470...",3,60.0,60.0,60.0,0.00000,0.7,2020-09-07T21:54:58Z,Jalan Tun Abdul Razak


In [121]:
print(len(here_traffic_flow_df))

353400


## Converting it to GDF

In [122]:
here_traffic_flow_gdf = gpd.GeoDataFrame(here_traffic_flow_df, geometry='Geometry')

## Adding addtional columns

In [123]:
here_traffic_flow_gdf['endpoints'] = here_traffic_flow_gdf['Geometry'].apply(lambda x: x.boundary)

def return_start_point(x):
    #print(x)
    try:
        return x.boundary[0]
    except:
        return None

def return_end_point(x):
    try:
        return x.boundary[1]
    except:
        return None
    

here_traffic_flow_gdf['start_point'] = here_traffic_flow_gdf['Geometry'].apply(lambda x: return_start_point(x))
here_traffic_flow_gdf['end_point'] = here_traffic_flow_gdf['Geometry'].apply(lambda x: return_end_point(x))

In [124]:
here_traffic_flow_gdf.head()

,TMC_PC,TMC_DE,TMC_QD,TMC_LE,Geometry,FC,CF_SP,CF_SU,CF_FF,CF_JF,CF_CN,PBT,DE,endpoints,start_point,end_point
0,4632,Jalan Tun Perak,-,0.05737,"LINESTRING (102.24085 2.21693, 102.24079 2.216...",2,43.0,43.0,43.7,0.21357,0.7,2020-09-07T21:59:58Z,Jalan Bachang,"MULTIPOINT (102.24085 2.21693, 102.24103 2.21715)",POINT (102.24085 2.21693),POINT (102.24103 2.21715)
1,4633,Lebuh AMJ,-,0.32415,"LINESTRING (102.24103 2.21715, 102.24110 2.217...",2,35.0,35.0,38.5,0.17316,0.7,2020-09-07T21:59:58Z,Jalan Bachang,"MULTIPOINT (102.24103 2.21715, 102.24173 2.21992)",POINT (102.24103 2.21715),POINT (102.24173 2.21992)
2,4627,Jalan Ayer Leleh/Jalan Laksamana Cheng Ho/Loro...,-,0.10352,"LINESTRING (102.26436 2.19317, 102.26459 2.193...",2,34.0,34.0,37.7,0.24756,0.7,2020-09-07T21:59:58Z,Jalan Semabok,"MULTIPOINT (102.26436 2.19317, 102.26528 2.19328)",POINT (102.26436 2.19317),POINT (102.26528 2.19328)
3,4628,Jalan Masjid Ujong Pasir,-,0.08944,"LINESTRING (102.26528 2.19328, 102.26554 2.193...",2,38.0,38.0,41.3,0.09685,0.7,2020-09-07T21:59:58Z,Jalan Semabok,"MULTIPOINT (102.26528 2.19328, 102.26606 2.19339)",POINT (102.26528 2.19328),POINT (102.26606 2.19339)
4,4630,Jalan Taman Tam Khor Nam,-,0.45397,"LINESTRING (102.26606 2.19339, 102.26619 2.193...",2,49.0,49.0,52.2,0.05108,0.7,2020-09-07T21:59:58Z,Jalan Semabok,"MULTIPOINT (102.26606 2.19339, 102.26889 2.19609)",POINT (102.26606 2.19339),POINT (102.26889 2.19609)


## Getting Intersection (Only routes within kl n selangor omit if not needed)

In [125]:
gadm = gpd.read_file('D:/Datasets/GeoSpatial/gadm_data/gadm36_MYS_1.shp')
kl_selangor_gadm = gadm[gadm['NAME_1'].isin(['Kuala Lumpur', 'Selangor'])]
kl_selangor_gadm.head()

,GID_0,NAME_0,GID_1,NAME_1,VARNAME_1,NL_NAME_1,TYPE_1,ENGTYPE_1,CC_1,HASC_1,geometry
3,MYS,Malaysia,MYS.4_1,Kuala Lumpur,Federal Territory of Kuala Lumpur,None,Wilayah Persekutuan,Federal Territory,None,MY.KL,"POLYGON ((101.73227 3.03781, 101.70284 3.03007..."
14,MYS,Malaysia,MYS.15_1,Selangor,Selangor Darul Ehsan,None,Negeri,State,None,MY.SL,"MULTIPOLYGON (((101.76600 2.86978, 101.74870 2..."


In [126]:
import geopandas as gpd
#g1 = gpd.GeoDataFrame.from_file("crime_stat.shp")
#g2 = gpd.GeoDataFrame.from_file("population.shp")
data = []
for index, gadm in kl_selangor_gadm.iterrows():
    for index2, traffic in here_traffic_flow_gdf.iterrows():
        if gadm['geometry'].intersects(traffic['Geometry']):
            data.append({'geometry': traffic['Geometry'].intersection(gadm['geometry']),
                        'TMC_PC': traffic['TMC_PC'],
                        'TMC_DE': traffic['TMC_DE'],
                         'TMC_QD': traffic['TMC_QD'],
                         'TMC_LE': traffic['TMC_LE'],
                        'FC': traffic['FC'],
                         'CF_SP': traffic['CF_SP'],
                         'CF_SU': traffic['CF_SU'],
                         'CF_FF': traffic['CF_FF'],
                         'CF_JF': traffic['CF_JF'],
                         'CF_CN': traffic['CF_CN'],
                         'PBT': traffic['PBT'],
                         'DE': traffic['DE'],
                         'endpoints': traffic['endpoints'],
                         'start_point': traffic['start_point'],
                         'end_point': traffic['end_point']
                        })

final_df = gpd.GeoDataFrame(data,columns=['geometry', 'TMC_PC', 'TMC_DE','TMC_QD', 'TMC_LE', 'FC', 
                                    'CF_SP', 'CF_SU', 'CF_FF', 'CF_JF', 'CF_CN', 'PBT', 'DE', 'endpoints', 'start_point', 'end_point' ])
#df.to_file('intersection.shp')
# control of the results in mi case, first values


In [127]:
final_df.head() # image from a Jupiter/IPython notebook

,geometry,TMC_PC,TMC_DE,TMC_QD,TMC_LE,FC,CF_SP,CF_SU,CF_FF,CF_JF,CF_CN,PBT,DE,endpoints,start_point,end_point
0,"LINESTRING (101.74195 3.08229, 101.74197 3.082...",5802,E37/Lebuhraya Hubungan Timur-Barat/Lebuhraya S...,-,0.71360,2,70.00,70.00,70.0,0.00000,0.70,2020-09-07T21:59:58Z,Lebuhraya Cheras-Kajang/Lebuhraya CKE/Cheras-K...,"(POINT (101.74195 3.08229), POINT (101.74658 3...",POINT (101.74195 3.08229),POINT (101.74658 3.07899)
1,"LINESTRING (101.74658 3.07899, 101.74688 3.078...",5803,Jalan Cheras Hartamas,-,0.11864,2,70.00,70.00,70.0,0.00000,0.70,2020-09-07T21:59:58Z,Lebuhraya Cheras-Kajang/Lebuhraya CKE/Cheras-K...,"(POINT (101.74658 3.07899), POINT (101.74763 3...",POINT (101.74658 3.07899),POINT (101.74763 3.0788)
2,"LINESTRING (101.74763 3.07880, 101.74779 3.078...",5804,Jalan Medan Masria,-,1.14547,2,84.00,84.00,90.0,0.97777,0.70,2020-09-07T21:59:58Z,Lebuhraya Cheras-Kajang/Lebuhraya CKE/Cheras-K...,"(POINT (101.74763 3.0788), POINT (101.75745 3....",POINT (101.74763 3.0788),POINT (101.75745 3.07577)
3,"LINESTRING (101.74829 3.07853, 101.74802 3.078...",5803,Jalan Cheras Hartamas,+,0.71240,2,67.49,67.49,84.2,2.91118,0.79,2020-09-07T21:59:58Z,Lebuhraya Cheras-Kajang/Lebuhraya CKE/Cheras-K...,"(POINT (101.7533 3.07699), POINT (101.74715 3....",POINT (101.7533 3.07699),POINT (101.74715 3.07869)
4,"LINESTRING (101.74715 3.07869, 101.74685 3.078...",5802,E37/Lebuhraya Hubungan Timur-Barat/Lebuhraya S...,+,0.82177,2,60.08,60.08,67.6,1.63232,0.77,2020-09-07T21:59:58Z,Lebuhraya Cheras-Kajang/Lebuhraya CKE/Cheras-K...,"(POINT (101.74715 3.07869), POINT (101.74182 3...",POINT (101.74715 3.07869),POINT (101.74182 3.08248)


In [128]:
final_df.count()

geometry       122688
TMC_PC         122688
TMC_DE         122688
TMC_QD         122688
TMC_LE         122688
FC             122688
CF_SP          122688
CF_SU          122688
CF_FF          122688
CF_JF          122688
CF_CN          122688
PBT            122688
DE             122688
endpoints      122688
start_point    122160
end_point      122160
dtype: int64

## Save as shp

In [14]:
pwd = 'C:\\Users\\tanji\\Desktop\\MIMOS_Geospatial'
try:
    os.mkdir('traffic_flow_shp/traffic_flow_26082020')
except:
    pass
#os.mkdir(pwd + '/traffic_flow/traffic_flow_26082020/')
here_traffic_flow_gdf.to_file('traffic_flow_shp/traffic_flow_26082020/traffic_flow_26082020_0700_0800.shp')

## Save as csv

In [129]:
final_df.to_csv('D:/Datasets/MIMOS/kl_selangor_traffic_flow_traffic_flow_07092020_2100_2200.csv', index=False)